## Set up

In [1]:
import pandas as pd
import re
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import pos_tag
from gensim.models import Phrases
from gensim.models.phrases import Phraser 
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.models.coherencemodel import CoherenceModel
from pprint import pprint
import pickle

In [2]:
data = pd.read_csv("data/cleaned_data1.csv")

/var/folders/5q/yq3hk8g1793ckqmn2n3xpr6c0000gn/T/ipykernel_2136/1751319474.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("data/cleaned_data1.csv")


## Preprocessing

Tokenizer

In [3]:
# Split the documents into tokens.
tokenizer = RegexpTokenizer(r'\w+')
data["tokenized"] = data["cleaned_text"].apply(lambda x: tokenizer.tokenize(x))

Stop words

In [4]:
#Custom defined list
english = [
    "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "as", "at", "be", 
    "because", "been", "before", "being", "below", "between", "both", "but", "by", "can", "cannot", "could", "did", 
    "do", "does", "doing", "down", "during", "each", "few", "for", "from", "further", "had", "has", "have", "having", 
    "he", "her", "here", "hers", "herself", "him", "himself", "his", "how", "i", "if", "in", "into", "is", "it", 
    "its", "itself", "let", "me", "more", "most", "must", "my", "myself", "no", "nor", "not", "of", "off", "on", 
    "once", "only", "or", "other", "our", "ours", "ourselves", "out", "over", "own", "same", "she", "some", "such", 
    "than", "that", "the", "their", "theirs", "them", "themselves", "then", "there", "these", "they", "this", "those", 
    "through", "to", "too", "under", "until", "up", "very", "was", "we", "were", "what", "when", "where", "which", 
    "while", "who", "whom", "why", "with", "would", "you", "your", "yours", "yourself", "yourselves", "will", "ll", 
    "re", "ve", "d", "s", "m", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", 
    "s", "t", "u", "v", "w", "x", "y", "z", "many", "us", "ok", "hows", "ive", "ill", "im", "cant", "topics", "topic",
    "discuss", "thoughts", "yo", "thats", "whats", "lets", "nothing", "oh", "omg", 
         "things", "stuff", "yall", "haha", "yes", "no", "wo", "like", 'good', 
         'work', 'got', 'going', 'dont', 'really', 'want', 'make', 'think', 
         'know', 'feel', 'people', 'life', "getting", "lot" "great", "i", "me", 
         "my", "myself", "we", "our", "ours", "ourselves", 
        "you", "your", "yours", "yourself", "yourselves", "he", "him", "his", 
        "himself", "she", "her", "hers", "herself", "it", "its", "itself", 
        "they", "them", "their", "theirs","themselves", "what", "which", "who", 
        "whom", "this", "that", "these", "those", "am", "is", "are", "was", 
        "were", "be", "been", "being", "have", "has", "had", "having", "do", 
        "does", "did", "doing", "will", "would", "should", "can", "could", "may",
        "might", "must", "shall", "ought", "about", "above", "across", "after", 
        "against", "along", "amid", "among", "around", "as", "at", "before", "behind",
        "below", "beneath", "beside", "between", "beyond", "but", "by", 
        "concerning", "considering", "despite", "down", "during", "except", "for",
        "from", "in", "inside", "into", "like", "near", "next", "notwithstanding",
        "of", "off", "on", "onto", "opposite", "out", "outside", "over", "past",
        "regarding", "round", "since", "than", "through", "throughout", "till", 
        "to", "toward", "towards", "under", "underneath", "unlike", "until", "up",
        "upon", "versus", "via", "with", "within", "without", "cant", "cannot", 
        "couldve", "couldnt", "didnt", "doesnt", "dont", "hadnt", "hasnt", 
        "havent", "hed", "hell", "hes", "howd", "howll", "hows", "id", "ill", 
        "im", "ive", "isnt", "itd", "itll", "its", "lets", "mightve", "mustve", 
        "mustnt", "shant", "shed", "shell", "shes", "shouldve", "shouldnt", 
        "thatll", "thats", "thered", "therell", "therere", "theres", "theyd", 
        "theyll", "theyre", "theyve", "wed", "well", "were", "weve", "werent", 
        "whatd", "whatll", "whatre", "whats", "whatve", "whend", "whenll", 
        "whens", "whered", "wherell", "wheres", "whichd", "whichll", "whichre", 
        "whichs", "whod", "wholl", "whore", "whos", "whove", "whyd", "whyll", 
        "whys", "wont", "wouldve", "wouldnt", "youd", "youll", "youre", "youve",
        "f", "m", "because", "go", "lot", "get", "still", "way", "something", "much",
        "thing", "someone", "person", "anything", "goes", "ok", "so", "just", "mostly", 
        "put", "also", "lots", "yet", "ha", "etc", "wasnt", "yeah", "okay", "lol"]

time = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", 
        "sunday", "morning", "noon", "afternoon", "evening", "night", "midnight",
        "dawn", "dusk", "week", "weekend", "weekends","weekly", "today", 
        "yesterday", "tomorrow", "yesterdays", "todays", "mondays", "tuesdays",
        "wednesdays", "thursdays", "fridays", "saturdays", "sundays", "day",
        "everyday", "daily", "workday", 'time', 'month', 'year', 'pm', 'am', "ago",
        "year"]

reddit = ["welcome", "hi", "hello", "sub", "reddit", "thanks", "thank", "maybe",
          "wo30", "mods", "mod", "moderators", "subreddit", "btw", "aw", "aww", 
          "aww", "hey", "hello", "join", "joined", "post", "op"]

topic_specific = ["self", "improvement", "selfimprovement", "rselfimprovement"]

stop_words = english + time + reddit + topic_specific

In [5]:
#Remove stopwords
def remove_stopwords_numbers(tokens):
    """Removes stopwords and tokens that contain any digits"""
    return [token for token in tokens if token not in stop_words and not re.search(r'\d', token)]

# Apply function to column
data["filtered_tokens"] = data["tokenized"].apply(remove_stopwords_numbers)

In [6]:
lemmatizer = WordNetLemmatizer()

def lemmatize_tokens(tokens):
    """Lemmatizes a list of tokens"""
    return [lemmatizer.lemmatize(token) for token in tokens]

# Apply function to column
data["lemmatized"] = data["filtered_tokens"].apply(lemmatize_tokens)

In [7]:
#https://www.h2kinfosys.com/blog/part-of-speech-tagging-chunking-with-nltk/
def get_wordnet_pos(tag):
    """Convert POS tag to a format recognized by WordNetLemmatizer"""
    if tag.startswith('J'): #NLTK tags adjectives as JJ (normal), 
                            #JJR (comparative) and JJS (superlative)
        return wordnet.ADJ  #and then it can be mapped to the wordnet database
                              #to get the lemma
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return None

def lemmatize_tokens(tokens):
    """Lemmatizes tokens with POS tagging"""
    tagged_tokens = pos_tag(tokens)  
    return [lemmatizer.lemmatize(word, pos) for word, tag in tagged_tokens
            if (pos := get_wordnet_pos(tag))]

data["lemmatized"] = data["filtered_tokens"].apply(lemmatize_tokens)

Bigrams

In [8]:
# Train bigram model on the lemmatized column
bigram = Phrases(data["lemmatized"], min_count=20)
bigram_phraser = Phraser(bigram)  # Optimizes since it is a lighter-weight
                                   #version of Phrases

def add_bigrams(tokens):
    """Adds bigrams to a list of tokens if they appear frequently enough."""
    bigram_tokens = bigram_phraser[tokens]
    return tokens + [token for token in bigram_tokens if '_' in token]

data["ngrams"] = data["lemmatized"].apply(add_bigrams)

Filtering of too rare of too common words

In [9]:
# Convert df column to a list of lists for later steps
docs = data["ngrams"].tolist() 

# Create a dictionary from the tokenized documents
dictionary = Dictionary(docs)

# Filter out words that appear in fewer than 20 documents or 
#  more than 80% of documents, and keep the 1500 most frequent
dictionary.filter_extremes(no_below=20, no_above=0.8, keep_n=1500)


Vectorize

In [10]:
# Create bag of words representation
corpus = [dictionary.doc2bow(doc) for doc in docs]

Save to disk

In [14]:
# Save corpus and dictionary together
with open('corpus_and_dict.pkl', 'wb') as f:
    pickle.dump((corpus, dictionary), f)


## Training

### Test 12

In [11]:
#Set hyperparameters
num_topics = 13
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model12 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model12.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model12, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model12.save("lda_models/model12") #CHANGE TWICE

[([(0.026085388, 'even'),
   (0.023647511, 'now'),
   (0.014701838, 'never'),
   (0.014698585, 'bad'),
   (0.014453678, 'well'),
   (0.014396181, 'right'),
   (0.014312367, 'hard'),
   (0.013488694, 'year'),
   (0.013176618, 'first'),
   (0.013120013, 'always'),
   (0.012973648, 'back'),
   (0.0107611185, 'try'),
   (0.010572806, 'best'),
   (0.009695846, 'actually'),
   (0.009158077, 'everything'),
   (0.009147877, 'problem'),
   (0.008958885, 'sure'),
   (0.00895677, 'long'),
   (0.00868239, 'care'),
   (0.008471456, 'happy')],
  -1.7960740884115263),
 ([(0.01964953, 'change'),
   (0.019536639, 'others'),
   (0.013981652, 'different'),
   (0.013427693, 'experience'),
   (0.012916323, 'world'),
   (0.010180403, 'mind'),
   (0.010141884, 'great'),
   (0.010083522, 'important'),
   (0.009129533, 'learn'),
   (0.008927057, 'focus'),
   (0.008769861, 'help'),
   (0.008264896, 'situation'),
   (0.008201473, 'need'),
   (0.008141944, 'instead'),
   (0.007886275, 'part'),
   (0.007419442, 'i

### Test 13

In [12]:
#Set hyperparameters
num_topics = 13
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 0.05
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model13 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model13.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model13, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model13.save("lda_models/model13") #CHANGE

[([(0.043877747, 'now'),
   (0.023260407, 'year'),
   (0.021478001, 'right'),
   (0.017157784, 'back'),
   (0.017040342, 'never'),
   (0.01606437, 'even'),
   (0.011340484, 'hard'),
   (0.01126541, 'well'),
   (0.011092494, 'right_now'),
   (0.01085999, 'best'),
   (0.010020439, 'relationship'),
   (0.009957139, 'bad'),
   (0.0098613445, 'always'),
   (0.00945056, 'happy'),
   (0.00826095, 'everything'),
   (0.008125086, 'first'),
   (0.007778453, 'long'),
   (0.0077713453, 'ever'),
   (0.00752067, 'man'),
   (0.0071542324, 'love')],
  -1.8445981066761983),
 ([(0.018778386, 'even'),
   (0.015371481, 'bad'),
   (0.013597275, 'everyone'),
   (0.012961145, 'problem'),
   (0.01270735, 'others'),
   (0.011692751, 'else'),
   (0.011021227, 'wrong'),
   (0.010237317, 'actually'),
   (0.009232475, 'always'),
   (0.009050164, 'advice'),
   (0.008849071, 'mean'),
   (0.008639003, 'anyone'),
   (0.008444035, 'comment'),
   (0.008369105, 'never'),
   (0.008363124, 'point'),
   (0.00819565, 'reason

### Test 14

In [13]:
#Set hyperparameters
num_topics = 13
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05
alpha = 0.08
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model14 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model14.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model14, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model14.save("lda_models/model14") #CHANGE

[([(0.026458358, 'now'),
   (0.02135575, 'even'),
   (0.020795163, 'never'),
   (0.01779167, 'bad'),
   (0.015127499, 'back'),
   (0.014539553, 'year'),
   (0.013507942, 'hard'),
   (0.013290656, 'well'),
   (0.012017829, 'always'),
   (0.011522531, 'everything'),
   (0.010299587, 'shit'),
   (0.009712711, 'ever'),
   (0.009119701, 'happy'),
   (0.008798005, 'best'),
   (0.008777562, 'right'),
   (0.008610539, 'man'),
   (0.008594, 'else'),
   (0.0070024026, 'everyone'),
   (0.006965686, 'love'),
   (0.0064846943, 'long')],
  -1.8976934616400705),
 ([(0.054802503, 'goal'),
   (0.039518032, 'right'),
   (0.038305815, 'now'),
   (0.036652893, 'step'),
   (0.026051313, 'small'),
   (0.023349179, 'change'),
   (0.022640241, 'start'),
   (0.021636156, 'right_now'),
   (0.020742329, 'first'),
   (0.016491653, 'long'),
   (0.012971996, 'hard'),
   (0.011223473, 'term'),
   (0.011171747, 'progress'),
   (0.01107305, 'plan'),
   (0.01091801, 'best'),
   (0.010894695, 'well'),
   (0.010749155, '

### Test 15

In [14]:
#Set hyperparameters
num_topics = 14
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05
alpha = 'auto'
eta = 0.5

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model15 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model15.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model15, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model15.save("lda_models/model15") #CHANGE

[([(0.033435963, 'even'),
   (0.030955855, 'now'),
   (0.019245295, 'never'),
   (0.018690221, 'hard'),
   (0.017732024, 'bad'),
   (0.017679397, 'year'),
   (0.016982917, 'back'),
   (0.016383752, 'right'),
   (0.01623709, 'always'),
   (0.01244335, 'long'),
   (0.011772219, 'first'),
   (0.0115627395, 'everything'),
   (0.011554778, 'relationship'),
   (0.011378834, 'well'),
   (0.011089149, 'happy'),
   (0.00935595, 'little'),
   (0.009086278, 'old'),
   (0.00844047, 'away'),
   (0.008312413, 'time'),
   (0.0082950285, 'love')],
  -1.874390542271013),
 ([(0.014528172, 'help'),
   (0.014048203, 'others'),
   (0.012290766, 'need'),
   (0.012196078, 'best'),
   (0.011893021, 'try'),
   (0.011798958, 'change'),
   (0.0108510135, 'problem'),
   (0.010058616, 'different'),
   (0.010049956, 'great'),
   (0.009655079, 'experience'),
   (0.009287095, 'world'),
   (0.009265625, 'look'),
   (0.009237755, 'everyone'),
   (0.008575118, 'else'),
   (0.008228296, 'part'),
   (0.008162629, 'actuall

### Test 16

In [25]:
#Set hyperparameters
num_topics = 13
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 0.05
eta = 0.05

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model16 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model16.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model16, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model16.save("lda_models/model16") #CHANGE TWICE

[([(0.043417767, 'now'),
   (0.022942975, 'year'),
   (0.021502854, 'right'),
   (0.01691441, 'never'),
   (0.01687377, 'back'),
   (0.015859868, 'even'),
   (0.01146093, 'best'),
   (0.011403229, 'well'),
   (0.011375713, 'hard'),
   (0.011046785, 'right_now'),
   (0.010194908, 'relationship'),
   (0.009889006, 'bad'),
   (0.009787713, 'always'),
   (0.009489121, 'happy'),
   (0.008223137, 'everything'),
   (0.008158369, 'first'),
   (0.007777415, 'ever'),
   (0.007692631, 'long'),
   (0.007605111, 'man'),
   (0.0072825863, 'love')],
  -1.8471516032913327),
 ([(0.018852532, 'even'),
   (0.015380959, 'bad'),
   (0.013676643, 'everyone'),
   (0.012921901, 'problem'),
   (0.0125369225, 'others'),
   (0.011715965, 'else'),
   (0.011001757, 'wrong'),
   (0.010228309, 'actually'),
   (0.009207175, 'always'),
   (0.009019651, 'advice'),
   (0.008880103, 'mean'),
   (0.00865535, 'anyone'),
   (0.00853929, 'comment'),
   (0.008363287, 'point'),
   (0.0083304085, 'never'),
   (0.008240015, 'rea

### Test 17

In [26]:
#Set hyperparameters
num_topics = 13
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 0.05
eta = 0.05

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model17 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model17.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model17, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model17.save("lda_models/model17") #CHANGE TWICE

[([(0.043417767, 'now'),
   (0.022942975, 'year'),
   (0.021502854, 'right'),
   (0.01691441, 'never'),
   (0.01687377, 'back'),
   (0.015859868, 'even'),
   (0.01146093, 'best'),
   (0.011403229, 'well'),
   (0.011375713, 'hard'),
   (0.011046785, 'right_now'),
   (0.010194908, 'relationship'),
   (0.009889006, 'bad'),
   (0.009787713, 'always'),
   (0.009489121, 'happy'),
   (0.008223137, 'everything'),
   (0.008158369, 'first'),
   (0.007777415, 'ever'),
   (0.007692631, 'long'),
   (0.007605111, 'man'),
   (0.0072825863, 'love')],
  -1.8471516032913327),
 ([(0.018852532, 'even'),
   (0.015380959, 'bad'),
   (0.013676643, 'everyone'),
   (0.012921901, 'problem'),
   (0.0125369225, 'others'),
   (0.011715965, 'else'),
   (0.011001757, 'wrong'),
   (0.010228309, 'actually'),
   (0.009207175, 'always'),
   (0.009019651, 'advice'),
   (0.008880103, 'mean'),
   (0.00865535, 'anyone'),
   (0.00853929, 'comment'),
   (0.008363287, 'point'),
   (0.0083304085, 'never'),
   (0.008240015, 'rea

### Test 18

In [27]:
#Set hyperparameters
num_topics = 13
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 0.1
eta = 0.1

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model18 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model18.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model18, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model18.save("lda_models/model18") #CHANGE TWICE

[([(0.054537807, 'year'),
   (0.03509401, 'now'),
   (0.027654301, 'old'),
   (0.020019043, 'never'),
   (0.01854134, 'back'),
   (0.018532218, 'young'),
   (0.018284716, 'parent'),
   (0.017843934, 'friend'),
   (0.017142875, 'relationship'),
   (0.01713591, 'age'),
   (0.016221937, 'kid'),
   (0.016100869, 'family'),
   (0.01167846, 'even'),
   (0.011441233, 'child'),
   (0.010393915, 'month'),
   (0.01037483, 'new'),
   (0.010187462, 'happy'),
   (0.009846569, 'long'),
   (0.009163218, 'last'),
   (0.008767541, 'best')],
  -1.952482741774617),
 ([(0.03428203, 'now'),
   (0.031830207, 'right'),
   (0.021204619, 'even'),
   (0.019218074, 'bad'),
   (0.015553466, 'hard'),
   (0.01476677, 'well'),
   (0.0143026, 'never'),
   (0.01342452, 'shit'),
   (0.0127655035, 'everything'),
   (0.012739384, 'else'),
   (0.012392274, 'right_now'),
   (0.010320881, 'always'),
   (0.010218537, 'change'),
   (0.010006084, 'everyone'),
   (0.009433718, 'back'),
   (0.008412197, 'gonna'),
   (0.007968025

### Test 19

In [28]:
#Set hyperparameters
num_topics = 13
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 0.1
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model19 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model19.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model19, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model19.save("lda_models/model19") #CHANGE TWICE

[([(0.04856775, 'year'),
   (0.03368064, 'now'),
   (0.028036304, 'old'),
   (0.019747835, 'never'),
   (0.019309012, 'friend'),
   (0.018751087, 'young'),
   (0.018688258, 'relationship'),
   (0.018435657, 'parent'),
   (0.018149957, 'back'),
   (0.01731045, 'age'),
   (0.01673341, 'family'),
   (0.016329201, 'kid'),
   (0.011610219, 'child'),
   (0.0115196975, 'even'),
   (0.010497275, 'new'),
   (0.010453972, 'happy'),
   (0.010203899, 'month'),
   (0.00978739, 'long'),
   (0.008881363, 'best'),
   (0.008877141, 'last')],
  -1.955458304337152),
 ([(0.03508529, 'now'),
   (0.0314958, 'right'),
   (0.021247208, 'even'),
   (0.018988889, 'bad'),
   (0.0156246, 'hard'),
   (0.0147718685, 'well'),
   (0.014460365, 'never'),
   (0.013314077, 'shit'),
   (0.012771586, 'everything'),
   (0.012290327, 'right_now'),
   (0.012130085, 'else'),
   (0.010371079, 'always'),
   (0.010272106, 'change'),
   (0.0097727515, 'back'),
   (0.009547905, 'everyone'),
   (0.008504121, 'best'),
   (0.00832423

### Test 20

In [29]:
#Set hyperparameters
num_topics = 13
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 0.1

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model20 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model20.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model20, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model20.save("lda_models/model20") #CHANGE TWICE

[([(0.026645463, 'even'),
   (0.024155214, 'now'),
   (0.015017486, 'never'),
   (0.015014169, 'bad'),
   (0.01475406, 'well'),
   (0.014718584, 'right'),
   (0.01461966, 'hard'),
   (0.013781732, 'year'),
   (0.013540661, 'first'),
   (0.013401705, 'always'),
   (0.01325219, 'back'),
   (0.01099272, 'try'),
   (0.010804526, 'best'),
   (0.009891645, 'actually'),
   (0.0093542775, 'everything'),
   (0.009288779, 'problem'),
   (0.009198712, 'long'),
   (0.009151285, 'sure'),
   (0.008868802, 'care'),
   (0.008653303, 'happy')],
  -1.797181490844846),
 ([(0.01971868, 'change'),
   (0.019609094, 'others'),
   (0.014034956, 'different'),
   (0.013477493, 'experience'),
   (0.012964226, 'world'),
   (0.010426216, 'mind'),
   (0.010195646, 'great'),
   (0.010187961, 'important'),
   (0.0091521675, 'learn'),
   (0.009132337, 'focus'),
   (0.008864407, 'help'),
   (0.008696993, 'situation'),
   (0.008224343, 'need'),
   (0.008212582, 'instead'),
   (0.007809562, 'part'),
   (0.0074488427, 'id

## Min probability tweaking

### Test 21

In [11]:
#Set hyperparameters
num_topics = 13
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.01 
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model21 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model21.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model21, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model21.save("lda_models/model21") #CHANGE TWICE

[([(0.026085388, 'even'),
   (0.023647511, 'now'),
   (0.014701838, 'never'),
   (0.014698585, 'bad'),
   (0.014453678, 'well'),
   (0.014396181, 'right'),
   (0.014312367, 'hard'),
   (0.013488694, 'year'),
   (0.013176618, 'first'),
   (0.013120013, 'always'),
   (0.012973648, 'back'),
   (0.0107611185, 'try'),
   (0.010572806, 'best'),
   (0.009695846, 'actually'),
   (0.009158077, 'everything'),
   (0.009147877, 'problem'),
   (0.008958885, 'sure'),
   (0.00895677, 'long'),
   (0.00868239, 'care'),
   (0.008471456, 'happy')],
  -1.7960740884115263),
 ([(0.01964953, 'change'),
   (0.019536639, 'others'),
   (0.013981652, 'different'),
   (0.013427693, 'experience'),
   (0.012916323, 'world'),
   (0.010180403, 'mind'),
   (0.010141884, 'great'),
   (0.010083522, 'important'),
   (0.009129533, 'learn'),
   (0.008927057, 'focus'),
   (0.008769861, 'help'),
   (0.008264896, 'situation'),
   (0.008201473, 'need'),
   (0.008141944, 'instead'),
   (0.007886275, 'part'),
   (0.007419442, 'i

### Test 22

In [12]:
#Set hyperparameters
num_topics = 13
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.07
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model22 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model22.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model22, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model22.save("lda_models/model22") #CHANGE TWICE

[([(0.026085388, 'even'),
   (0.023647511, 'now'),
   (0.014701838, 'never'),
   (0.014698585, 'bad'),
   (0.014453678, 'well'),
   (0.014396181, 'right'),
   (0.014312367, 'hard'),
   (0.013488694, 'year'),
   (0.013176618, 'first'),
   (0.013120013, 'always'),
   (0.012973648, 'back'),
   (0.0107611185, 'try'),
   (0.010572806, 'best'),
   (0.009695846, 'actually'),
   (0.009158077, 'everything'),
   (0.009147877, 'problem'),
   (0.008958885, 'sure'),
   (0.00895677, 'long'),
   (0.00868239, 'care'),
   (0.008471456, 'happy')],
  -1.7960740884115263),
 ([(0.01964953, 'change'),
   (0.019536639, 'others'),
   (0.013981652, 'different'),
   (0.013427693, 'experience'),
   (0.012916323, 'world'),
   (0.010180403, 'mind'),
   (0.010141884, 'great'),
   (0.010083522, 'important'),
   (0.009129533, 'learn'),
   (0.008927057, 'focus'),
   (0.008769861, 'help'),
   (0.008264896, 'situation'),
   (0.008201473, 'need'),
   (0.008141944, 'instead'),
   (0.007886275, 'part'),
   (0.007419442, 'i

### Test 23

In [13]:
#Set hyperparameters
num_topics = 13
chunksize = 2000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.1  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model23 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model23.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model23, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model23.save("lda_models/model23") #CHANGE TWICE

[([(0.026085388, 'even'),
   (0.023647511, 'now'),
   (0.014701838, 'never'),
   (0.014698585, 'bad'),
   (0.014453678, 'well'),
   (0.014396181, 'right'),
   (0.014312367, 'hard'),
   (0.013488694, 'year'),
   (0.013176618, 'first'),
   (0.013120013, 'always'),
   (0.012973648, 'back'),
   (0.0107611185, 'try'),
   (0.010572806, 'best'),
   (0.009695846, 'actually'),
   (0.009158077, 'everything'),
   (0.009147877, 'problem'),
   (0.008958885, 'sure'),
   (0.00895677, 'long'),
   (0.00868239, 'care'),
   (0.008471456, 'happy')],
  -1.7960740884115263),
 ([(0.01964953, 'change'),
   (0.019536639, 'others'),
   (0.013981652, 'different'),
   (0.013427693, 'experience'),
   (0.012916323, 'world'),
   (0.010180403, 'mind'),
   (0.010141884, 'great'),
   (0.010083522, 'important'),
   (0.009129533, 'learn'),
   (0.008927057, 'focus'),
   (0.008769861, 'help'),
   (0.008264896, 'situation'),
   (0.008201473, 'need'),
   (0.008141944, 'instead'),
   (0.007886275, 'part'),
   (0.007419442, 'i

## Chunksize 

### Test 24

In [14]:
#Set hyperparameters
num_topics = 13
chunksize = 1500 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model24 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model24.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model24, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model24.save("lda_models/model24") #CHANGE TWICE

[([(0.041786417, 'now'),
   (0.02706168, 'friend'),
   (0.025755776, 'never'),
   (0.025617238, 'right'),
   (0.025353381, 'well'),
   (0.025019525, 'hard'),
   (0.024069995, 'best'),
   (0.02286102, 'back'),
   (0.02230172, 'always'),
   (0.02113818, 'year'),
   (0.020156335, 'relationship'),
   (0.01841197, 'bad'),
   (0.017486453, 'new'),
   (0.017269151, 'even'),
   (0.016040474, 'everything'),
   (0.015233601, 'care'),
   (0.014970367, 'happy'),
   (0.014196771, 'long'),
   (0.013399595, 'love'),
   (0.011415708, 'away')],
  -1.7935312071840817),
 ([(0.020034958, 'even'),
   (0.016566914, 'help'),
   (0.014462032, 'first'),
   (0.013982601, 'try'),
   (0.013877749, 'need'),
   (0.012360732, 'problem'),
   (0.0117969215, 'actually'),
   (0.011055686, 'sure'),
   (0.0109649105, 'different'),
   (0.010731451, 'look'),
   (0.0105618695, 'great'),
   (0.010544054, 'everyone'),
   (0.010272701, 'enough'),
   (0.010028433, 'point'),
   (0.009916849, 'talk'),
   (0.009862332, 'else'),
   

### Test 25

In [15]:
#Set hyperparameters
num_topics = 13
chunksize = 1000 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model25 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model25.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model25, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model25.save("lda_models/model25") #CHANGE TWICE

[([(0.029922314, 'even'),
   (0.027100297, 'now'),
   (0.016815763, 'bad'),
   (0.016727403, 'never'),
   (0.016706329, 'well'),
   (0.016552685, 'right'),
   (0.016458673, 'first'),
   (0.016285647, 'hard'),
   (0.015114853, 'always'),
   (0.014820778, 'back'),
   (0.014052866, 'best'),
   (0.01293083, 'try'),
   (0.011043015, 'new'),
   (0.011042558, 'actually'),
   (0.010969818, 'long'),
   (0.010360613, 'everything'),
   (0.010348161, 'sure'),
   (0.0098496005, 'year'),
   (0.009611237, 'enough'),
   (0.0087195765, 'look')],
  -1.76990222425558),
 ([(0.019288475, 'help'),
   (0.018914467, 'change'),
   (0.018621298, 'others'),
   (0.015549614, 'need'),
   (0.013380811, 'different'),
   (0.012985028, 'experience'),
   (0.012636086, 'world'),
   (0.012020761, 'everyone'),
   (0.011731531, 'happy'),
   (0.011479666, 'mind'),
   (0.010901727, 'love'),
   (0.01075942, 'situation'),
   (0.010279793, 'matter'),
   (0.010145554, 'problem'),
   (0.00936173, 'mean'),
   (0.009330071, 'import

### Test 26

In [16]:
#Set hyperparameters
num_topics = 13
chunksize = 2500 #to speed up training
passes = 20
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model26 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model26.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model26, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model26.save("lda_models/model26") #CHANGE TWICE

[([(0.039183076, 'now'),
   (0.03069217, 'even'),
   (0.0235323, 'never'),
   (0.023171427, 'right'),
   (0.0216548, 'back'),
   (0.021037195, 'hard'),
   (0.020881677, 'well'),
   (0.020531347, 'best'),
   (0.020511903, 'year'),
   (0.016107993, 'always'),
   (0.01607231, 'first'),
   (0.015303321, 'everything'),
   (0.014336335, 'long'),
   (0.011471294, 'old'),
   (0.011030109, 'little'),
   (0.0105740195, 'place'),
   (0.009465112, 'big'),
   (0.009228579, 'time'),
   (0.00917085, 'ever'),
   (0.00916025, 'away')],
  -1.8844372227057253),
 ([(0.018742671, 'bad'),
   (0.015201515, 'help'),
   (0.01463488, 'need'),
   (0.014384621, 'try'),
   (0.014160516, 'change'),
   (0.012494694, 'problem'),
   (0.012124031, 'different'),
   (0.011943284, 'actually'),
   (0.0117161665, 'everyone'),
   (0.010552049, 'look'),
   (0.010208788, 'advice'),
   (0.010129687, 'reason'),
   (0.009804489, 'else'),
   (0.009771507, 'point'),
   (0.009612026, 'part'),
   (0.009334185, 'kind'),
   (0.00923609

## Fine tuning passes

### Test 27

In [17]:
#Set hyperparameters
num_topics = 13
chunksize = 2500 #to speed up training
passes = 25
iterations = 100
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model27 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model27.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model27, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model27.save("lda_models/model27") #CHANGE TWICE

[([(0.039177235, 'now'),
   (0.031666495, 'even'),
   (0.023538366, 'never'),
   (0.023183685, 'right'),
   (0.021653526, 'back'),
   (0.021530434, 'well'),
   (0.021070458, 'hard'),
   (0.02052955, 'year'),
   (0.020332266, 'best'),
   (0.016981238, 'always'),
   (0.01632317, 'first'),
   (0.015411304, 'everything'),
   (0.014328312, 'long'),
   (0.01146872, 'old'),
   (0.011043388, 'little'),
   (0.010679829, 'place'),
   (0.009822957, 'big'),
   (0.009346277, 'away'),
   (0.009294428, 'time'),
   (0.009182476, 'ever')],
  -1.8843761204117293),
 ([(0.019240439, 'bad'),
   (0.015549931, 'help'),
   (0.014941397, 'need'),
   (0.014702503, 'try'),
   (0.01277242, 'problem'),
   (0.012389002, 'different'),
   (0.012219812, 'actually'),
   (0.011997035, 'everyone'),
   (0.011375693, 'change'),
   (0.010997675, 'look'),
   (0.010466147, 'advice'),
   (0.010337236, 'reason'),
   (0.010110812, 'else'),
   (0.010101067, 'point'),
   (0.009838861, 'part'),
   (0.00954105, 'kind'),
   (0.009441

## Fine tuning iterations 

### Test 28

In [18]:
#Set hyperparameters
num_topics = 13
chunksize = 2500 
passes = 25
iterations = 75 #prevent over fitting to small patterns, trying to reduce vague topics
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model28 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model28.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model28, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model28.save("lda_models/model28") #CHANGE TWICE

[([(0.025439668, 'even'),
   (0.01477182, 'bad'),
   (0.014684423, 'best'),
   (0.014276221, 'well'),
   (0.013593389, 'hard'),
   (0.012845933, 'always'),
   (0.011772692, 'first'),
   (0.011615903, 'help'),
   (0.011369842, 'need'),
   (0.01120341, 'try'),
   (0.009656235, 'problem'),
   (0.009639659, 'everything'),
   (0.009238459, 'actually'),
   (0.009072313, 'everyone'),
   (0.008874557, 'sure'),
   (0.008710636, 'look'),
   (0.008496209, 'different'),
   (0.008178544, 'care'),
   (0.008028317, 'enough'),
   (0.007917077, 'advice')],
  -1.9413427679416677),
 ([(0.076668814, 'now'),
   (0.046063982, 'never'),
   (0.042375375, 'back'),
   (0.04017585, 'year'),
   (0.032783136, 'right'),
   (0.022444084, 'old'),
   (0.017958406, 'ever'),
   (0.017660776, 'live'),
   (0.017551227, 'age'),
   (0.017067118, 'man'),
   (0.016239049, 'young'),
   (0.015827572, 'happy'),
   (0.014264751, 'kid'),
   (0.014124029, 'right_now'),
   (0.013864545, 'shit'),
   (0.012872859, 'alone'),
   (0.0128

### Test 29

In [15]:
#Set hyperparameters
num_topics = 11
chunksize = 2500 
passes = 25
iterations = 75 
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model29 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model29.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model29, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model29.save("lda_models/model29") #CHANGE TWICE

[([(0.022791298, 'even'),
   (0.022597294, 'now'),
   (0.013576855, 'never'),
   (0.013347966, 'right'),
   (0.012889879, 'bad'),
   (0.012489685, 'back'),
   (0.012153377, 'hard'),
   (0.011977151, 'well'),
   (0.011480834, 'always'),
   (0.01121363, 'first'),
   (0.010759099, 'friend'),
   (0.0106348945, 'year'),
   (0.01022161, 'best'),
   (0.008788148, 'everything'),
   (0.008608632, 'try'),
   (0.008155955, 'relationship'),
   (0.007951627, 'long'),
   (0.007934427, 'sure'),
   (0.0078192605, 'happy'),
   (0.0078105326, 'actually')],
  -1.7500492364204474),
 ([(0.019605413, 'change'),
   (0.017853862, 'others'),
   (0.014085087, 'world'),
   (0.012065873, 'experience'),
   (0.010923082, 'different'),
   (0.010606526, 'mind'),
   (0.0094396155, 'instead'),
   (0.008893179, 'feeling'),
   (0.008826107, 'important'),
   (0.00849515, 'focus'),
   (0.008340795, 'situation'),
   (0.0077292803, 'control'),
   (0.0068959896, 'human'),
   (0.006866163, 'positive'),
   (0.0068496196, 'value

### Test 30

In [16]:
#Set hyperparameters
num_topics = 12
chunksize = 2500 
passes = 25
iterations = 75 
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model30 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model30.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model30, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model30.save("lda_models/model30") #CHANGE TWICE

[([(0.027463017, 'even'),
   (0.027229311, 'now'),
   (0.01635985, 'never'),
   (0.016113337, 'right'),
   (0.01591412, 'bad'),
   (0.015865594, 'friend'),
   (0.015367166, 'well'),
   (0.015049828, 'back'),
   (0.014644586, 'hard'),
   (0.014208863, 'best'),
   (0.0141276, 'first'),
   (0.013845572, 'always'),
   (0.012199505, 'year'),
   (0.010812537, 'start'),
   (0.010724042, 'everything'),
   (0.010645424, 'long'),
   (0.010290676, 'new'),
   (0.010275607, 'relationship'),
   (0.009427303, 'happy'),
   (0.009381374, 'need')],
  -1.7160375336726756),
 ([(0.026362639, 'help'),
   (0.020163393, 'problem'),
   (0.019529866, 'different'),
   (0.017696982, 'talk'),
   (0.016014067, 'advice'),
   (0.014905186, 'situation'),
   (0.014681321, 'kind'),
   (0.0136652, 'issue'),
   (0.013587761, 'mean'),
   (0.012493776, 'idea'),
   (0.012479591, 'wrong'),
   (0.011220171, 'reason'),
   (0.010958158, 'actually'),
   (0.010531881, 'ask'),
   (0.010511818, 'less'),
   (0.009988068, 'question'),

### Test 31

In [18]:
#Set hyperparameters
num_topics = 12
chunksize = 2500 
passes = 25
iterations = 70 
eval_every = None
minimum_probability = 0.05  
alpha = 'auto'
eta = 'auto'

# Make an index to word dictionary.
temp = dictionary[0]  
id2word = dictionary.id2token

#Run  #CHANGE FIRST LINE
model31 = LdaModel( 
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha=alpha,
    eta=eta,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every,
    random_state = 21,
    minimum_probability=minimum_probability
)

#Evaluate
top_topics = model31.top_topics(corpus)  #CHANGE
pprint(top_topics)
cm = CoherenceModel(model=model31, corpus=corpus, texts=docs, coherence='c_v', dictionary=dictionary) #CHANGE
print(cm.get_coherence())  
model31.save("lda_models/model31") #CHANGE TWICE

[([(0.029024528, 'even'),
   (0.02877751, 'now'),
   (0.017290039, 'never'),
   (0.017029515, 'right'),
   (0.016818965, 'bad'),
   (0.016250148, 'well'),
   (0.016001252, 'friend'),
   (0.015905533, 'back'),
   (0.015477253, 'hard'),
   (0.015316738, 'best'),
   (0.014632795, 'always'),
   (0.013422478, 'year'),
   (0.012384153, 'first'),
   (0.01132575, 'everything'),
   (0.010717801, 'long'),
   (0.010692126, 'relationship'),
   (0.009466637, 'happy'),
   (0.009335733, 'look'),
   (0.009311975, 'care'),
   (0.009140919, 'enough')],
  -1.8000350663983073),
 ([(0.04816591, 'goal'),
   (0.046427704, 'start'),
   (0.031442016, 'book'),
   (0.030409416, 'step'),
   (0.029044116, 'new'),
   (0.028442541, 'small'),
   (0.017911596, 'learn'),
   (0.016907101, 'easy'),
   (0.015875451, 'great'),
   (0.0152604375, 'skill'),
   (0.014409632, 'read'),
   (0.013745186, 'term'),
   (0.01366061, 'first'),
   (0.013646599, 'use'),
   (0.013219013, 'practice'),
   (0.012693379, 'free'),
   (0.012564